In [1]:
import pandas as pd
import xarray as xr
import numpy as np

import re
import datetime
import time

import os
from collections import OrderedDict

import matplotlib.pyplot as plt
import matplotlib

from sys import getsizeof,path
path.append("../src")
#from Utility import DateFrom8digitName,RKI_AG_to_int

In [2]:
# Landkreis adjacency
lkadj = {} 
# SH, Hamburg
lkadj[1001] = [1059,"DK"] # SK Flensbug
lkadj[1002] = [1057,1058] # Kiel
lkadj[1003] = [1053,1055,1062,13074] # Lübeck
lkadj[1004] = [1057,1058,1060] # Neumünster
lkadj[1051] = [1054,1059,1057,1061,3359,3352] # Dithmarschen
lkadj[1053] = [1003,1062,2000,3353,3355,13074,13076] # Herzogtum Lauenburg
lkadj[1054] = [1051,1059,"DK"] # Nordfriesland
lkadj[1055] = [1003,1058,1060,1062] # Ostholstein
lkadj[1056] = [1060,1061,2000,3359] # Pinneberg
lkadj[1057] = [1051,1058,1059,1060,1061,1002,1004] # Rendsburg-Eckernförde
lkadj[1058] = [1002,1004,1055,1057,1060] # Plön
lkadj[1059] = [1001,1051,1054,1057,"DK"] # Schleswig-Flensburg
lkadj[1060] = [1004,1055,1056,1057,1058,1061,1062,2000] # Segeberg
lkadj[1061] = [1051,1056,1057,1060,3359] # Steinburg
lkadj[1062] = [1003,1053,1055,1060,2000] # Stormarn
lkadj[2000] = [1053,1056,1060,1062,3353,3359] # Hamburg
# NDS, Bremen
lkadj[3101] = [3101,3151,3154,3157,3158] # SK Braunschweig
lkadj[3102] = [3102,3153,3157,3158,3254] # SK Salzgitter
lkadj[3103] = [3154,3151] # SK Wolfsburg
lkadj[3151] = [3101,3103,3141,3157,3154,3351,3360,15081,15083] # LK Gifhorn
lkadj[3153] = [3102,3155,3158,3159,3254,15085,16062] # LK Goslar
lkadj[3154] = [3101,3103,3151,3158,15083,15085] # LK Helmstedt
lkadj[3155] = [3153,3159,3254,3255,6633,5762] # LK Northeim
lkadj[3157] = [3101,3102,3151,3241,3254] # LK Peine
lkadj[3158] = [3101,3102,3153,3154,3254,15085] # LK Wolfenbüttel
lkadj[3159] = [3153,3155,6611,6633,6636,16061,16062] # LK Göttingen
lkadj[3241] = [3151,3157,3351,3352,3354,3356,3357,3358] # Region Hannover
lkadj[3251] = [3256,3361,4011,3401,3458,3460,3459,5770] # LK Diepholz
lkadj[3252] = [3241,3257,5766,3255,3254] # LK Hameln-Pyrmont
lkadj[3254] = [3102,3153,3155,3157,3158,3255,3252] # LK Hildesheim
lkadj[3255] = [3155,3252,3254,5766,5762] # LK Holzminden
lkadj[3256] = [3241,3251,3358,3361,5770,3257] # LK Nienburg (Weser)
lkadj[3257] = [3241,3252,3256,5766,5770] # LK Schaumburg
lkadj[3351] = [3151,3241,3360,3358] # LK Celle
lkadj[3352] = [1051,3359,4012,3461,3356,3357] # Cuxhaven
lkadj[3353] = [2000,3355,3357,3358,1053] # Harburg
lkadj[3354] = [15081,15090,12070,3355,3360,13076] # LK Lüchow-Dannenberg
lkadj[3355] = [1053,3353,13076,3354,3358,3360] # Lüneburg
lkadj[3356] = [3352,3461,4011,3361,3357] # LK Osterholz
lkadj[3357] = [3351,3352,3353,3356,3358,3359,3361] # LK Rotenburg (Wümme)
lkadj[3358] = [3241,3256,3351,3353,3355,3357,3361,3360] # LK Heidekreis
lkadj[3359] = [1051,1056,1061,2000,3352,3357] # Stade
lkadj[3360] = [3151,3351,3354,3358,15081] # LK Uelzen
lkadj[3361] = [3251,3256,3356,3357,3358,4011] # LK Verden
lkadj[3401] = [3251,4011,3461,3458] # SK Delmenhorst
lkadj[3402] = [3452,3457,"NL"] # SK Emden
lkadj[3403] = [3458,3461,3451] # SK Oldenburg
lkadj[3404] = [3459,5566] # SK Osnabrück
lkadj[3405] = [3461,3455] # SK Wilhelmshaven
lkadj[3451] = [3403,3461,3458,3457,3453] # LK Ammerland
lkadj[3452] = [3402,3457,3462,"NL"] # LK Aurich
lkadj[3453] = [3451,3458,3460,3459,3454,3457] # LK Cloppenburg
lkadj[3454] = [3453,3457,3459,5566,3456,"NL"] # LK Emsland
lkadj[3455] = [3405,3462,3457] # LK Friesland
lkadj[3456] = [3454,"NL",5566,5554] # LK Grafschaft Bentheim
lkadj[3457] = [3402,3451,3452,3453,3454,3455,3462,"NL"] # LK Leer
lkadj[3458] = [3251,3401,3403,3451,3453,3460,3461] # LK Oldenburg
lkadj[3459] = [3251,3404,3453,3454,5566,5570,5754,5758,5770,3460] # LK Osnabrück
lkadj[3460] = [3251,3453,3458,3459] # LK Vechta
lkadj[3461] = [3352,3356,3401,3403,3405,3451,3458,4012,4011] # LK Wesermarsch
lkadj[3462] = [3452,3455,3457] # LK Wittmund
# NRW & Bremen
lkadj[4011] = [3251,3356,3361,3401,3461] # SK Bremen
lkadj[4012] = [3352,3461] # SK Bremerhaven
lkadj[5111] = [5112,5162,5158] # SK Düsseldorf
lkadj[5112] = [5111,5170,5114,5158,5117,5119,5162] # SK Duisburg
lkadj[5113] = [5562,5512,5119,5117,5158,5954,5911,5513] # SK Essen
lkadj[5114] = [5112,5170,5166,5162] # SK Krefeld
lkadj[5116] = [5166,5370,5162] # SK Mönchengladbach
lkadj[5117] = [5112,5113,5158,5119] # SK Mülheim a.d.Ruhr
lkadj[5119] = [5112,5113,5117,5170,5512] # SK Oberhausen
lkadj[5120] = [5122,5124,5378,5374] # SK Remscheid
lkadj[5122] = [5120,5124,5158,5378] # SK Solingen
lkadj[5124] = [5120,5122,5158,5374,5954] # SK Wuppertal
lkadj[5154] = [5554,5170,5166,"NL"] # LK Kleve
lkadj[5158] = [5111,5112,5113,5117,5122,5124,5162,5315,5316,5378,5954] # LK Mettmann
lkadj[5162] = [5111,5112,5114,5116,5162,5166,5370,5358,5362,5315] # LK Rhein-Kreis Neuss
lkadj[5166] = [5114,5116,5154,5162,5370,5170,"NL"] # LK Viersen
lkadj[5170] = [5112,5114,5119,5154,5166,5554,5562,5512] # LK Wesel
lkadj[5314] = [5382,7131] # SK Bonn
lkadj[5315] = [5158,5162,5362,5382,5378,5316] # SK Köln
lkadj[5316] = [5158,5315,5378] # SK Leverkusen
lkadj[5334] = [5370,5358,5366,"BE"] # StadtRegion Aachen
lkadj[5358] = [5162,5334,5370,5366,5362] # LK Düren
lkadj[5362] = [5162,5315,5358,5382,5366] # LK Rhein-Erft-Kreis
lkadj[5366] = [5334,5358,5362,5382,7131,7233,7232,"BE"] # LK Euskirchen
lkadj[5370] = [5116,5162,5166,5334,5358,"NL"] # LK Heinsberg
lkadj[5374] = [5120,5124,5378,5382,7132,5966,5962,5954] # LK Oberbergischer Kreis
lkadj[5378] = [5120,5122,5158,5315,5316,5374,5382] # LK Rheinisch-Bergischer Kreis
lkadj[5382] = [5314,5315,5362,5366,5374,5378,7131,7138,7132] # LK Rhein-Sieg-Kreis
lkadj[5512] = [5113,5119,5170,5562] # SK Bottrop
lkadj[5513] = [5113,5911,5916,5562] # SK Gelsenkirchen
lkadj[5515] = [5558,5566,5570] # SK Münster
lkadj[5554] = [3456,5154,5170,5566,5558,5562,"NL"] # LK Borken
lkadj[5558] = [5515,5554,5562,5978,5915,5570,5566] # LK Coesfeld
lkadj[5562] = [5113,5170,5512,5513,5554,5558,5916,5911,5913,5978] # LK Recklinghausen
lkadj[5566] = [3404,3454,3456,3459,5515,5554,5558,5570] # LK Steinfurt
lkadj[5570] = [3459,5515,5558,5566,5915,5974,5754] # LK Warendorf
lkadj[5711] = [5758,5754,5766] # SK Bielefeld
lkadj[5754] = [3459,5570,5711,5974,5774,5766,5758] # LK Gütersloh
lkadj[5758] = [3459,5711,5754,5770,5766] # LK Herford
lkadj[5762] = [3255,5766,5774,5958,6635,6633,3155] # LK Höxter
lkadj[5766] = [3252,3255,3257,5711,5754,5758,5762,5774,5770] # LK Lippe
lkadj[5770] = [3251,3256,3257,3459,5758,5766] # LK Minden-Lübbecke
lkadj[5774] = [5754,5762,5766,5958,5974] # LK Paderborn
lkadj[5911] = [5113,5513,5562,5916,5913,5954] # SK Bochum
lkadj[5913] = [5562,5911,5954,5914,5978] # SK Dortmund
lkadj[5914] = [5913,5954,5962,5978] # SK Hagen
lkadj[5915] = [5558,5570,5978,5974] # SK Hamm
lkadj[5916] = [5513,5562,5911] # SK Herne
lkadj[5954] = [5113,5124,5158,5374,5911,5913,5914,5962] # LK Ennepe-Ruhr-Kreis
lkadj[5958] = [5762,5774,5974,5962,5966,5970,6635] # LK Hochsauerlandkreis
lkadj[5962] = [5914,5374,5954,5958,5970,5966,5974,5978] # LK Märkischer Kreis
lkadj[5966] = [5374,5374,5958,5962] # LK Olpe
lkadj[5970] = [5962,5958,7132,7143,6532,6534,6635] # LK Siegen-Wittgenstein
lkadj[5974] = [5570,5754,5774,5915,5958,5962,5978] # LK Soest
lkadj[5978] = [5558,5562,5913,5914,5915,5962,5974] # LK Unna
# Hessen
lkadj[6411] = [6432,6438] # SK Darmstadt
lkadj[6412] = [6434,6436,6433,6438,6413,6435,6440] # SK Frankfurt am Main
lkadj[6413] = [6412,6435,6438] # SK Offenbach
lkadj[6414] = [6439,7339,7315,6433,6436] # SK Wiesbaden

lkadj[6431] = [] # LK Bergstraße

lkadj[6432] = [6411] # LK Darmstadt-Dieburg
lkadj[6433] = [6412,6414] # LK Groß-Gerau
lkadj[6434] = [6412] # LK Hochtaunuskreis
lkadj[6435] = [6412,6413] # LK Main-Kinzig-Kreis
lkadj[6436] = [6412,6414] # LK Main-Taunus-Kreis
lkadj[6437] = [] # LK Odenwaldkreis
lkadj[6438] = [6411,6412,6413] # LK Offenbach
lkadj[6439] = [6414] # LK Rheingau-Taunus-Kreis
lkadj[6440] = [6412] # LK Wetteraukreis
lkadj[6531] = [] # LK Gießen
lkadj[6532] = [5970] # LK Lahn-Dill-Kreis
lkadj[6533] = [] # LK Limburg-Weilburg
lkadj[6534] = [5970] # LK Marburg-Biedenkopf
lkadj[6535] = [] # LK Vogelsbergkreis
lkadj[6611] = [3155,3159] # SK Kassel
lkadj[6631] = [] # LK Fulda
lkadj[6632] = [] # LK Hersfeld-Rotenburg
lkadj[6633] = [6611,3159,5762] # LK Kassel
lkadj[6634] = [] # LK Schwalm-Eder-Kreis
lkadj[6635] = [5762,5958,5970] # LK Waldeck-Frankenberg
lkadj[6636] = [3159,16061] # LK Werra-Meißner-Kreis
# RLP
lkadj[7111] = [] # SK Koblenz
lkadj[7131] = [5314,5366,5382] # LK Ahrweiler
lkadj[7132] = [5374,5382,5970] # LK Altenkirchen
lkadj[7133] = [] # LK Bad Kreuznach
lkadj[7134] = [] # LK Birkenfeld
lkadj[7135] = [] # LK Cochem-Zell
lkadj[7137] = [] # LK Mayen-Koblenz
lkadj[7138] = [5382] # LK Neuwied
lkadj[7140] = [] # LK Rhein-Hunsrück-Kreis
lkadj[7141] = [] # LK Rhein-Lahn-Kreis
lkadj[7143] = [5970] # LK Westerwaldkreis
lkadj[7211] = [] # SK Trier
lkadj[7231] = [] # LK Bernkastel-Wittlich
lkadj[7232] = [5366] # LK Bitburg-Prüm
lkadj[7233] = [5366] # LK Vulkaneifel
lkadj[7235] = [] # LK Trier-Saarburg
lkadj[7311] = [] # SK Frankenthal
lkadj[7312] = [] # SK Kaiserslautern
lkadj[7313] = [] # SK Landau i.d.Pfalz
lkadj[7314] = [] # SK Ludwigshafen
lkadj[7315] = [6414] # SK Mainz
lkadj[7316] = [] # SK Neustadt a.d.Weinstraße
lkadj[7317] = [] # SK Pirmasens
lkadj[7318] = [] # SK Speyer
lkadj[7319] = [] # SK Worms
lkadj[7320] = [] # SK Zweibrücken
lkadj[7331] = [] # LK Alzey-Worms
lkadj[7332] = [] # LK Bad Dürkheim
lkadj[7333] = [] # LK Donnersbergkreis
lkadj[7334] = [] # LK Germersheim
lkadj[7335] = [] # LK Kaiserslautern
lkadj[7336] = [] # LK Kusel
lkadj[7337] = [] # LK Südliche Weinstraße
lkadj[7338] = [] # LK Rhein-Pfalz-Kreis
lkadj[7339] = [6414] # LK Mainz-Bingen
lkadj[7340] = [] # LK Südwestpfalz

lkadj[9661] = [9671] # SK Aschaffenburg
lkadj[9671] = [9661] #LK Aschaffenburg'

lkadj[12070] = [3354,13076] # LK Prignitz

lkadj[13074] = [1003,13076] # Nordwestmecklenburg
lkadj[13076] = [1053,13074,12070,3354,3355] # Ludwigslust-Parchim
lkadj[15081] = [3151,3354,3360] # LK Altmarkkreis Salzwedel
lkadj[15083] = [3151,3154] # LK Börde

lkadj[15090] = [3354] # LK Stendal
lkadj[15091] = [] # LK Wittenberg


lkadj[15085] = [3153,3154,3158] # LK Harz

lkadj[16061] = [6636,3159,16062] # LK Eichsfeld
lkadj[16062] = [3153,3159,16061] # LK Nordhausen

In [ ]:
vk = [k for k,v in lkadj.items() if len(v) > 0]
vk = [k for k in vk if int(k/1000) == 5 ]
c = np.zeros((len(vk),len(vk)))

print(len(vk),vk)

for i,k in enumerate(vk):
    for k2 in lkadj[k]:
        if k2 in vk:
            j = vk.index(k2)
            c[i,j] = 1
            c[j,i] = 1

m = 1000
pos = np.random.rand(len(vk),2)*2
o = np.zeros((m+1,pos.shape[0],pos.shape[1]))
imp = np.zeros_like(pos)
o[0] = pos

for n in range(m):
    ij = 0
    potential = 0
    for i in range(0,len(vk)):
        f = np.zeros(2)
        for j in range(0,len(vk)):
            if i != j:
                a,b = pos[i],pos[j]
                d = (b-a)
                dd = np.sqrt((d*d).sum())-1
                if c[i,j] > 0:
                    potential += dd*dd
                if dd < 1:
                    f -= d*(n/m)
                f += d*dd*c[i,j]*(1-(n/m))
        imp[i] = imp[i]/10.+f*.1
    if n%50 == 0:
        print(n,potential)
    pos += imp
    o[n+1] = pos

colors = ["blue","red","green","purple","orange"]
ax = plt.subplot(111)
for i,k in enumerate(vk):
    color = colors[int(k/1000)-1]
    c = plt.Circle((o[-1,i,0],o[-1,i,1],),1.,alpha=.1,color=color)
    ax.text(o[-1,i,0],o[-1,i,1],k)
    ax.add_patch(c)
 #   if i%1 == 0:
#        ax.plot(o[:,i,0],o[:,i,1],'o',label=k,alpha=.05,color=color)
#ax.legend()
ax.set(xlim=(np.min(o[-1,:]-1),np.max(o[-1,:])+1,), ylim = (np.min(o[-1,:]-1),np.max(o[-1,:])+1,))

for i,k in enumerate(vk):
    v = lkadj[k]
    for e in v:
        if e in vk:
            j = vk.index(e)
            ax.plot((o[-1,i,0],o[-1,j,0],),(o[-1,i,1],o[-1,j,1],),"k",alpha=.1)

In [3]:
class ForceLayoutGraph(object):
    def __init__(self):
        nodes = {} 
    
    def AddNode(self,node):
        pass
    
class ForceLayoutNode(object):
    _counter = 0
    def __init__(self,name=None,properties=None):
        self.id = ForceLayoutNode._counter
        ForceLayoutNode._counter += 1
        if name == None:
            self.name = "node_%d"%self.id
        else:
            self.name = name
        if properties == None:
            self.properties = {}
        else:
            self.properties = properties
    
     
    
class GeoForceLayoutNode(ForceLayoutNode):
    def __init__(self,name=None):
        super(GeoForceLayoutNode,self).__init__(name)
        
    
    




In [ ]:
nodes = {}
nodes['A'] = ['B','C']
nodes['B'] = ['A','C','D']
nodes['C'] = ['A','B','E']
nodes['D'] = ['B','E','F']
nodes['E'] = ['C','D','F','G']
nodes['F'] = ['D','E','G']
nodes['G'] = ['E','F']

print(nodes)

vk = [k for k,v in nodes.items() if len(v) > 0]
c = np.zeros((len(vk),len(vk)))

for i,k in enumerate(vk):
    for k2 in nodes[k]:
        if k2 in vk:
            j = vk.index(k2)
            c[i,j] = 1
            c[j,i] = 1
            
m = 200
pos = np.random.rand(len(vk),2)*4
o = np.zeros((m+1,pos.shape[0],pos.shape[1]))
imp = np.zeros_like(pos)
o[0] = pos

for n in range(m):
    ij = 0
    potential = 0
    for i in range(0,len(vk)):
        f = np.zeros(2)
        for j in range(0,len(vk)):
#        for j in range(i+1,len(vk)):
            if i != j:
                a,b = pos[i],pos[j]
                d = (b-a)
                dd = np.sqrt((d*d).sum())-1
                if c[i,j] > 0:
                    potential += dd*dd
                if dd < 1:
                    f -= d*(n/m)
                f += d*dd*c[i,j]*(1-(n/m))
        imp[i] = imp[i]/10.+f*.1
    print(n,potential)
    pos += imp
    o[n+1] = pos


ax = plt.subplot(111)
for i,k in enumerate(vk):
    c = plt.Circle((o[-1,i,0],o[-1,i,1],),1.,alpha=.1)
    ax.add_patch(c)
    ax.plot(o[:,i,0],o[:,i,1],'o',label=k,alpha=.1)
ax.legend()

for i,v in enumerate(nodes.values()):
    for e in v:
        j = vk.index(e)
        ax.plot((o[-1,i,0],o[-1,j,0],),(o[-1,i,1],o[-1,j,1],),"k")
#ax.set(xlim=(np.min(o[-1,:]-1),np.max(o[-1,:])+1,), ylim = (np.min(o[-1,:]-1),np.max(o[-1,:])+1,))


In [11]:
def Partitiongraph(g):
    neighbour_id = {}
    p_hull,p_core,p_adj = {},{},{}
    
    for k,v in g.items():
        if len(v) > 0:
            kp = int(k//1000)*1000
            v = v.copy()
        # sieve out neighbouring countries and add them back with proper ID
            neighbours = [x for x in v if type(x) == str]
            for n in neighbours:
                v.pop(v.index(n))
                if n not in neighbour_id:
                    neighbour_id[n] = len(neighbour_id)
                nid = neighbour_id.get(n)
                v.append(nid)
                if nid not in p_adj:
                    p_adj[nid] = set()
        # determine bundesland neighbours from lk-id
            vp = set([int(x//1000)*1000 if x>1000 else x for x in v])
            lvp = len(vp)
            if lvp == 1 and kp in vp: # core
                if kp not in p_core:
                    p_core[kp] = {}
                p_core[kp][k] = v
            else: # hull, there are adjacent countries / bundesländer
                if kp not in p_hull:
                    p_hull[kp] = {}
                p_hull[kp][k] = v
            # adjacent clusters
                if kp not in p_adj:
                    p_adj[kp] = set()
                if kp in vp:
                    vp.remove(kp)
                p_adj[kp].update(vp)
            # update reverse adjacent
                for rvp in vp:
                    if rvp not in p_adj:
                        p_adj[rvp] = set()
                    p_adj[rvp].add(kp)
                
    return neighbour_id,p_hull,p_core,p_adj   
            
nids,hulls,cores,adjacent = Partitiongraph(lkadj)

print(adjacent,"\n")
print(hulls)

core 1000 1002 [1057, 1058] {1000} True
core 1000 1004 [1057, 1058, 1060] {1000} True
core 1000 1055 [1003, 1058, 1060, 1062] {1000} True
core 1000 1057 [1051, 1058, 1059, 1060, 1061, 1002, 1004] {1000} True
core 1000 1058 [1002, 1004, 1055, 1057, 1060] {1000} True
core 3000 3101 [3101, 3151, 3154, 3157, 3158] {3000} True
core 3000 3102 [3102, 3153, 3157, 3158, 3254] {3000} True
core 3000 3103 [3154, 3151] {3000} True
core 3000 3157 [3101, 3102, 3151, 3241, 3254] {3000} True
core 3000 3241 [3151, 3157, 3351, 3352, 3354, 3356, 3357, 3358] {3000} True
core 3000 3254 [3102, 3153, 3155, 3157, 3158, 3255, 3252] {3000} True
core 3000 3351 [3151, 3241, 3360, 3358] {3000} True
core 3000 3357 [3351, 3352, 3353, 3356, 3358, 3359, 3361] {3000} True
core 3000 3358 [3241, 3256, 3351, 3353, 3355, 3357, 3361, 3360] {3000} True
core 3000 3403 [3458, 3461, 3451] {3000} True
core 3000 3405 [3461, 3455] {3000} True
core 3000 3451 [3403, 3461, 3458, 3457, 3453] {3000} True
core 3000 3453 [3451, 3458, 3460

In [12]:
def HullPotential(ipos,ihull,shull):
    ds,n = 0,len(ihull)
    for k,v in shull.items():
        i = ipos[ihull[k]]
        dvs = 0
        for x in v:
            j = ipos[ihull[x]]
            d = min((i-j)%n,(j-i)%n) # ring metric
            dvs += d*d
        ds += dvs
    return ds

def OptimizeHull(ipos,ihull,shull):
    n = len(ipos)
    p_initial = HullPotential(ipos,ihull,shull)
    shifts,tries,tries_count,maxstep = 0,0,0,max(1,int(n/2))
    
    step = -maxstep
    upper = n*3
    while tries < upper:
        if step == -maxstep:
            shifts = 0
        for k in shull.keys():
            p0 = HullPotential(ipos,ihull,shull)
            ia = ipos[ihull[k]] # original position
            ib = ipos.index((ia+step)%n) # index of next position
            jpos = ipos.copy()
            jpos[ib] = (jpos[ib]-step)%n
            jpos[ihull[k]] = (ia+step)%n
            
            p1 = HullPotential(jpos,ihull,shull)
            if p1 < p0:
                shifts += 1
                ipos = jpos
        step = (step+maxstep+1)%(2*maxstep)-(maxstep)
        if step == 0:
            step = 1
        
        tries_count = tries
        tries += 1
        if shifts == 0 and step == 1:
            tries = upper
    # return order of elements on hull
    return ipos

def OrderedIntegerHull(hid,hull):
    shull = {}
    hkeys = list(hull.keys())
    for k,v in hull.items():
        h = [x for x in v if x >= hid and x <= hid+1000]
        h = [x for x in h if x in hkeys]
        shull[k] = h
    n = len(shull)
    
    if n > 3: # more than three element to position
        ihull = {}
        for i,k in enumerate(shull.keys()):
            ihull[k] = i
        ipos = list(range(n))
        ipos = OptimizeHull(ipos,ihull,shull)
 
        return ipos
    else:
        return list(range(n))
        
"""
       ax = plt.subplot(111)
        pos = np.zeros((n,2,))
        for j,i in enumerate(ipos):
            alpha = 2.*np.pi/n*i
            pos[j] = [np.sin(alpha),np.cos(alpha)]
        for i,k in enumerate(ihull.keys()):
            ax.text(pos[i,0],pos[i,1],(k,shull[k],))
            for k2 in shull[k]:
                ax.plot([pos[i,0],pos[ihull[k2],0]],[pos[i,1],pos[ihull[k2],1]])
"""

def ForceLayoutCore(hid,hull,ihull,core):
    # Layout the Hull
    nhull,ncore = len(hull),len(core)
    npos = nhull+ncore
    pos = np.zeros((npos,4,),dtype="float32")
    
    for j,i in enumerate(ihull):
        alpha = 2.*np.pi/nhull*i
        pos[j,:2] = [np.sin(alpha),np.cos(alpha)]

        
    print(core)
        
    # Build index for physical interaction
    index = dict(zip(list(hull.keys())+list(core.keys()),range(nhull+ncore)))
    pindex = []
    for i,k in enumerate(core.keys()):
        for k2 in core[k]:
            print(i,k,index[k],k2,index[k2])
    
    # try to pull core-nodes apart via hull-attraction, no collision between non-interacting nodes
    c_dist = np.sqrt(ncore)
    
    print(hid)
    
  #  print(hull)
  #  print(ihull)
  #  print(cores)
  #  print("")

for i,k in enumerate(hulls.keys()):
    hull = hulls[k]
  
    ihull = OrderedIntegerHull(k,hull)
    if k in cores:
        ForceLayoutCore(k,hull,ihull,cores[k])
    

{1002: [1057, 1058], 1004: [1057, 1058, 1060], 1055: [1003, 1058, 1060, 1062], 1057: [1051, 1058, 1059, 1060, 1061, 1002, 1004], 1058: [1002, 1004, 1055, 1057, 1060]}
0 1002 10 1057 13
0 1002 10 1058 14
1 1004 11 1057 13
1 1004 11 1058 14
1 1004 11 1060 7
2 1055 12 1003 1
2 1055 12 1058 14
2 1055 12 1060 7
2 1055 12 1062 9
3 1057 13 1051 2
3 1057 13 1058 14
3 1057 13 1059 6
3 1057 13 1060 7
3 1057 13 1061 8
3 1057 13 1002 10
3 1057 13 1004 11
4 1058 14 1002 10
4 1058 14 1004 11
4 1058 14 1055 12
4 1058 14 1057 13
4 1058 14 1060 7
1000
{3101: [3101, 3151, 3154, 3157, 3158], 3102: [3102, 3153, 3157, 3158, 3254], 3103: [3154, 3151], 3157: [3101, 3102, 3151, 3241, 3254], 3241: [3151, 3157, 3351, 3352, 3354, 3356, 3357, 3358], 3254: [3102, 3153, 3155, 3157, 3158, 3255, 3252], 3351: [3151, 3241, 3360, 3358], 3357: [3351, 3352, 3353, 3356, 3358, 3359, 3361], 3358: [3241, 3256, 3351, 3353, 3355, 3357, 3361, 3360], 3403: [3458, 3461, 3451], 3405: [3461, 3455], 3451: [3403, 3461, 3458, 3457, 345

6 6435 13 6412 8
6 6435 13 6413 9
7 6436 14 6412 8
7 6436 14 6414 0
8 6438 15 6411 7
8 6438 15 6412 8
8 6438 15 6413 9
9 6439 16 6414 0
10 6440 17 6412 8
6000
